In [142]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Linear regression
## Normal equation

In [143]:
housing  = pd.read_csv('/Users/zyan/program/ml/datasets/housing/housing.csv')

In [144]:
housing_target = housing['median_house_value']

In [145]:
housing = housing.drop(['median_house_value','ocean_proximity'],axis=1)

In [146]:
m , n = housing.shape

In [147]:
housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')

In [148]:
housing_plus_bias  = np.c_[np.ones((m,1)),housing]

In [149]:
X = tf.constant(housing_plus_bias,dtype=tf.float64,name='X')
Y = tf.constant(housing_target.values.reshape(-1,1),dtype=tf.float64,name='Y')

In [150]:
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),Y)

In [151]:
with tf.Session() as sess:
    theta_value = theta.eval()

## Gradient Descent

### Prepare data for gradient descent 

In [152]:
from sklearn.preprocessing import Imputer
from sklearn import preprocessing

In [153]:
imputer = Imputer(strategy='mean')

In [154]:
housing.shape

(20640, 8)

In [155]:
housing = imputer.fit_transform(housing)

In [156]:
housing_scaled = preprocessing.scale(housing)

In [157]:
housing_scaled_plus_bias = np.c_[np.ones((m,1)),housing_scaled]

In [158]:
n_epochs = 1000
learning_rate = 0.01

In [159]:
X = tf.constant(housing_scaled_plus_bias,dtype=tf.float32,name='X')
Y = tf.constant(housing_target.values.reshape(-1,1),dtype=tf.float32,name='Y')

In [160]:
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,-1.0),name='theta')

In [161]:
Y_pred = tf.matmul(X,theta,name = 'predictions')

In [162]:
error = Y-Y_pred
mse = tf.reduce_mean(tf.square(error),name='mse')
gradients = 2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta-learning_rate*gradients)

In [163]:
init = tf.global_variables_initializer()

In [164]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100 ==0:
            print('Epoch',epoch,'MSE= ',mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE=  56105464000.0
Epoch 100 MSE=  181823030000000.0
Epoch 200 MSE=  6.002448e+20
Epoch 300 MSE=  2.0112107e+27
Epoch 400 MSE=  6.738874e+33
Epoch 500 MSE=  inf
Epoch 600 MSE=  inf
Epoch 700 MSE=  inf
Epoch 800 MSE=  inf
Epoch 900 MSE=  inf



**You don't have to hard caculate gradient by yourself. Tensorflow can do it for you.**
```
gradients = tf.gradients(mse,[theta])[0]
```


In [197]:
x = tf.constant(1,dtype='float32')
y=tf.constant(2,dtype='float32')

In [198]:
z = x+2*y
gradients = tf.gradients(z,[y,x])

In [199]:
init1 = tf.global_variables_initializer()

In [200]:
with tf.Session() as sess:
    sess.run(init)
    print(gradients[1].eval())

1.0


**Then you can update z by its gradients**

### Furthermore, you can choose optimizer to compute the gradients

In [204]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# choose other different optimizer : optimizer = tf.train.MomentumOptimizer(...)
training_op = optimizer.minimize(mse)

### Feed data using mini-batch

In [205]:
X = tf.placeholder(tf.float32,shape=(None,n+1),name='X')# shape = None means any length of the dimention

In [206]:
Y = tf.placeholder(tf.float32,shape=(None,1),name='Y')

In [208]:
batch_size =100
n_batches = int(np.ceil(m/batch_size))

Then fetch mini-batch one by one and feed to the node

In [210]:
def fetch_batch(epoch,batch_index,batch_size):
    x_batch = housing_scaled_plus_bias[batch_index*batch_size:batch_index*batch_size+batch_size]
    y_batch = housing_target[batch_index*batch_size:batch_index*batch_size+batch_size]
    return x_batch,y_batch 

In [213]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch,y_batch = fetch_batch(epoch,batch_index,batch_size)
            sess.run(training_op,feed_dict={X:x_batch,Y:y_batch.values.reshape(-1,1)})

## Saving and restoring models

### Saving
Just create a Saver node at the end of the construction and in the execution phase, call its save() method whenever you want to save the model
```
...
...
saver = tf.train.Saver()
with tf.Session() as sess:
...
save_path = saver.save(sess,'/tmp/my_model.ckpt')
...
...
save_path = saver.save(sess,'/tmp/my_model_final.ckpt')

```

### Restoring

your create a Saver at the end of the construction and at the begining of execution,instead of initializing the variables using the init node , you can call the restore() method of the Saver
```
...
saver = tf.train.Saver()
with tf.Session() as sess:
saver.restore(sess,'/tmp/my_model_final.ckpt')
...
...
```

## Tensorboard
Use a different log directory every time you run your program so you won't mess up the tensor board when it load the log file. So you can include date in log directory name.<span style="color:red">Aboid logging training stats at every training step, as this would slow down training.</span>

**At the begining of the program**

In [215]:
from datetime import datetime

In [216]:
now = datetime.now().strftime('%Y%m%d%H%M%S')

In [217]:
root_logdir = 'tf_logs'
logdir = '{}/run-{}'.format(root_logdir,now)

Then add the following at the very end of construction 
```
mse_summary = tf.summary.scalar('MSE',mse) # this node will evaluate the MSE value and write it to a log string called summay
file_write = tf.summary.FileWrite(logdir,tf.get_default_graph()) 
```
Update the execution to evaluate mse_summary regularly during the training.

```
...
for bath_index in range(n_bathes):
    x_batch,y_batch...
    if batch_index %10==0:
        summary_str = mse_summary.eval(feed_dic={X:x_batch,Y:y_batch})
        step = epoch*n_batch+batch_index
        file_write.add_summary(summary_str,step)
    sess.run(training_op,feed_dict={X:x_batch,Y:y_batch})
...
```

At the end of program, close the FileWriter

```
file_write.colse()
```
To start TensorBoard web server point to the root log directory
```
tensorboard --logdir tf_logs
```

## Name space

In [220]:
with tf.name_scope('loss') as scope:
    error = Y_pred-Y
    mse = tf.reduce_mean(tf.square(error),name='mse')

In [222]:
print(error.op.name)

loss_1/sub


In [223]:
print(mse.op.name)

loss_1/mse


## Sharing Variable
use get_variable when you reuse the variable you have to set scope's reuse = True

In [225]:
with tf.variable_scope('relu'):
    threshold = tf.get_variable('threshold',shape=())

In [231]:
with tf.variable_scope('relu'):
    threshold = tf.get_variable('threshol',shape=())